In [2]:
import torch
import pandas as pd
import numpy as np
#import re
#import nltk
#nltk.download()

from transformers import *

In [3]:
ber = "text_sentences.txt"
data = pd.read_csv(ber, delimiter='\n', header=None)
data.head()

,0
0,alpha-catenin inhibits beta-catenin signaling ...
1,A binary complex of birch profilin and skeleta...
2,Abnormal immunoreactivity in the tumor tissue ...
3,Absence of alpha-syntrophin leads to structura...
4,"Abundance of actin , talin , alpha 5 and beta ..."


In [4]:
print('Total sentences :',len(data))
max(data[0].str.len())

Total sentences : 1100


646

In [5]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
def Bert_Tokens(sentence):
        tokenized = tokenizer.encode_plus(
                            text=sentence,  # the sentence to be encoded
                            add_special_tokens=True,  # Add [CLS] and [SEP]
                            max_length = 40,  # maximum length of a sentence
                            pad_to_max_length=True,  # Add [PAD]s
                            #truncation = True,
                        #     padding=True,
                            return_attention_mask = True,  # Generate the attention mask
                        #     return_tensors = 'pt',  # ask the function to return PyTorch tensors
                        )
        input_ids = torch.LongTensor([np.array(tokenized['input_ids'])])
        attention_mask = torch.LongTensor([np.array(tokenized['attention_mask'])])
        return input_ids,attention_mask

In [6]:
input_ids,attention_mask = Bert_Tokens(data[0][0])
print((input_ids.shape))
print(type(input_ids))
y = input_ids.squeeze(0)
print(y.shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


torch.Size([1, 40])
<class 'torch.Tensor'>
torch.Size([40])


/opt/anaconda3/envs/nlpenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2016: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [7]:
z = attention_mask.squeeze(0)
a=torch.sum(z)

In [8]:
ty = data[0][0].split()
print(ty)
print(len(ty))

['alpha-catenin', 'inhibits', 'beta-catenin', 'signaling', 'by', 'preventing', 'formation', 'of', 'a', 'beta-catenin*T-cell', 'factor*DNA', 'complex', '.']
13


In [9]:
text=data[0][0]
marked_text = "[CLS] " + text+ " [SEP]"
# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)
tt = tokenized_text
# Print out the tokens.
print(tokenized_text)
print(len(tokenized_text))

['[CLS]', 'alpha', '-', 'cat', '##enin', 'inhibits', 'beta', '-', 'cat', '##enin', 'signaling', 'by', 'preventing', 'formation', 'of', 'a', 'beta', '-', 'cat', '##enin', '*', 't', '-', 'cell', 'factor', '*', 'dna', 'complex', '.', '[SEP]']
30


In [10]:
def parse_bert(seq_original, seq_bert):
    """
    """

    def remove_leading_pounds(token):
        """
        """
        token_new = ''
        for c in token:
            if c != '#':
                token_new += c
        return token_new

    # Remove header and footer tags.
#     seq_bert = seq_bert[1:-1])
    # Iterate over the original sequence and detect splitted tokens.
    mapped_indices_list = []
    j = 0
    for i in range(len(seq_original)):
        if seq_original[i] == seq_bert[j]:  # Not splitted.
            j += 1
            continue
        else:  # Detect splitted tokens.
            start = 0
            token_splitted = seq_original[i]
            token_mapping = remove_leading_pounds(seq_bert[j])
            mapped_indices = []
            while token_mapping == \
                    token_splitted[start : start + len(token_mapping)]:
                mapped_indices.append(j)
                start += len(token_mapping)
                j += 1
                token_mapping = remove_leading_pounds(seq_bert[j])
            mapped_indices_list.append((i, mapped_indices))
    return mapped_indices_list


    
    

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
    'allenai/scibert_scivocab_uncased')

text_test = data[0][0]
seq_original = [w.lower() for w in data[0][0].split(' ')]
seq_bert = tokenizer.tokenize(text_test )
print("Original word sequence:", seq_original)
print(len(seq_original))
print("BERT word sequence:", seq_bert)
print(len(seq_bert))

splits = parse_bert(seq_original, seq_bert)
for split in splits:
    print("Splitted token:", seq_original[split[0]])
    print("Splitted to:",
        ', '.join([seq_bert[i] for i in split[1]]))
    print()
print((splits))

Original word sequence: ['alpha-catenin', 'inhibits', 'beta-catenin', 'signaling', 'by', 'preventing', 'formation', 'of', 'a', 'beta-catenin*t-cell', 'factor*dna', 'complex', '.']
13
BERT word sequence: ['alpha', '-', 'cat', '##enin', 'inhibits', 'beta', '-', 'cat', '##enin', 'signaling', 'by', 'preventing', 'formation', 'of', 'a', 'beta', '-', 'cat', '##enin', '*', 't', '-', 'cell', 'factor', '*', 'dna', 'complex', '.']
28
Splitted token: alpha-catenin
Splitted to: alpha, -, cat, ##enin

Splitted token: beta-catenin
Splitted to: beta, -, cat, ##enin

Splitted token: beta-catenin*t-cell
Splitted to: beta, -, cat, ##enin, *, t, -, cell

Splitted token: factor*dna
Splitted to: factor, *, dna

[(0, [0, 1, 2, 3]), (2, [5, 6, 7, 8]), (9, [15, 16, 17, 18, 19, 20, 21, 22]), (10, [23, 24, 25])]


In [12]:
def Bert_New_Embedding(bert,split,shape):
    '''
    '''
    print((split))
    j=0
    k=0
    bert_new=torch.zeros([1,shape,768])
    if len(split) != 0:
        for i in range(len(split)):
            while k< split[i][0]:
                bert_new[:,k,:]=bert[:,j,:]
                j += 1
                k += 1
            for p in range(len(split[i][1])):
                j += 1

            bert_new[:,k,:] = torch.sum(bert[:,split[i][1],:],dim = 1)/len(split[i][1])
            k +=1
        while k < shape:
            bert_new[:,k,:]=bert[:,j,:]
            j += 1
            k += 1
    else:
        bert_new = bert
        
    return bert_new

In [13]:
config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config)
with torch.no_grad():
    outputs1 = model(input_ids,attention_mask )

In [14]:
bert_out=outputs1[0]
bert_out = bert_out[:,0:a,:]
bert_out = bert_out[:,1:-1,:]
bert_out.shape

torch.Size([1, 28, 768])

In [15]:
bert_out = Bert_New_Embedding(bert_out,splits,len(seq_original))

[(0, [0, 1, 2, 3]), (2, [5, 6, 7, 8]), (9, [15, 16, 17, 18, 19, 20, 21, 22]), (10, [23, 24, 25])]


In [16]:
bert_out[0][10]

tensor([ 5.0830e-01,  1.3862e-01, -1.0345e+00, -5.0161e-01, -9.3059e-01,
         9.3089e-02,  5.6875e-01,  3.6135e-01, -6.9744e-01,  4.3429e-01,
         4.2599e-01,  2.0977e-02, -3.8835e-01,  3.2372e-02, -1.1121e+00,
        -8.8309e-01,  5.9325e-01,  6.4571e-01,  1.4261e+00,  3.5278e-01,
         6.8543e-02, -2.2690e-01,  5.4788e-02,  3.0494e-01, -5.1349e-01,
        -8.2899e-02,  1.5260e-01, -7.5267e-02, -4.1532e-01,  1.8757e-01,
         3.9923e-01,  4.6759e-01,  1.5293e-01, -1.0006e+00,  6.7551e-01,
         2.5596e-01,  5.6533e-01, -4.4140e-02,  1.1028e-02, -6.6146e-02,
        -5.1838e-01,  9.0952e-01,  1.2917e+00, -3.8168e-01,  6.5312e-01,
         2.9474e-02,  2.7219e-01, -2.5619e-01, -3.7606e-01, -7.6582e-01,
         3.6864e-01, -6.9876e-01, -4.5156e-01, -1.3754e-03, -3.7420e-01,
         2.5871e-01, -4.3326e-01,  6.2785e-01,  3.2660e-02, -8.9519e-01,
        -9.9723e-01,  2.1043e-02,  4.2238e-02,  1.7319e-01,  1.4061e+00,
         7.7810e-02,  3.1586e-02,  8.3118e-02,  1.9

In [17]:
bt = outputs1[0]
for i in range(3):
    sm = (bt[0][24][i]+bt[0][25][i]+bt[0][26][i])/3
    print(sm)
print(bt[0][27])
idx = [1, 2, 3, 4]
ad = bt[:,idx,:].sum()/len(idx)

tensor(0.5083)
tensor(0.1386)
tensor(-1.0345)
tensor([-1.2065e-01, -1.1065e+00, -2.5464e-01, -5.7854e-01, -3.2624e-01,
        -2.0647e-01,  7.0490e-01,  9.9409e-01, -6.3572e-01, -1.4784e-01,
         1.0825e+00, -7.5510e-01, -1.5567e-01, -6.7080e-01, -1.8279e+00,
        -2.4280e-01,  1.4367e+00,  8.8319e-02,  9.4183e-01,  4.0336e-01,
         1.6213e-01, -1.7464e-03,  1.0651e+00, -5.9033e-01,  3.9550e-01,
        -5.2400e-02,  1.0258e+00, -1.3702e-01, -5.4504e-01, -5.4080e-01,
        -1.2052e+00, -4.5433e-01,  3.7176e-02, -8.8752e-01,  7.3869e-01,
         2.2285e-01,  4.4025e-01,  1.3059e-01,  8.8312e-01, -5.7267e-01,
         2.1058e-01,  1.0572e-03,  1.1544e+00, -2.7692e-01,  2.3384e+00,
        -8.1864e-01,  9.5467e-01,  3.5287e-01, -8.5025e-01, -6.7571e-01,
         1.5203e-01, -1.2396e+00, -1.9128e-01, -2.9623e-02,  1.5094e-01,
        -2.2973e-01, -1.5330e+00, -5.1553e-01,  1.2345e-01, -5.7522e-01,
        -1.4762e+00,  6.9819e-01, -1.1420e-01,  6.3110e-01,  8.3623e-01,
     